In [5]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import norm
import random
import shapely
from shapely.geometry import LineString,Point,MultiLineString
import itertools
from numpy.linalg import multi_dot

In [6]:
# Funtion to generate the nodes and form a grid
# m, n = no of rows, no of columns
# y,x = spacing of rows, spacing of columns

def mesh(m,n,y,x):
    nodes = []
    for j in np.arange(0,m*y,y):
        #print(j)
        for k in np.arange(0,n*x,x):
            #print(k)
            nodes.append([k,j])
            #print(nodes[i])
    return nodes

# returns a list containing x,y cordinates of the nodes

In [7]:
# Function to separate active nodes and inactive nodes
# active_nodes = nodes connected to elements
# inactive_nodes = nodes not connected to any elements

def ActiveNodes(elements, totalnodes):
    active_nodes = np.unique(np.concatenate((elements[:,1], elements[:,2])))
    inactive_nodes = np.setdiff1d(np.arange(totalnodes), active_nodes)
    return sorted(active_nodes), inactive_nodes

In [8]:
# Function to compute the distance bewteen the nodes
# n = node number
# curr_nodes = nodes connected to an element

def computeDistance(n, active_nodes, curr_nodes, grid):
    avail = set(active_nodes) - set(curr_nodes)
    distanceArray = [[i, ((grid[n][0] - grid[i][0]) ** 2 + (grid[n][1] - grid[i][1]) ** 2) ** 0.5] for i in avail]
    return sorted(distanceArray, key=lambda x: x[1])

In [9]:
# Function to generate line elements 

def updateLines(elemat,grid):
    lines = [LineString([tuple(grid[i[1]]),tuple(grid[i[2]])]) for i in elemat]
    return lines

In [10]:
# Funtion to calculate the total volume of the line segments
# Volume = length x area =  length x 1 = length

def TrussVolume(elemat,grid):
    lines = updateLines(elemat,grid)
    network = MultiLineString(lines)
    return network.length

In [11]:
# Function to distribute the self weight of the elements to the nodes

def GetForces(a, P, n,):
    p = P / len(a)
    c = np.concatenate((a[:,1], a[:,2]))
    d, counts = np.unique(c, return_counts=True)
    forces = np.zeros((len(d), 4))
    forces[:, 0] = np.arange(1, len(d)+1)
    forces[:, 1] = d
    forces[:, 2] = 1
    forces[:, 3] = (p / 2) * counts
    i = np.where(forces[:, 1] == n)[0][0]
    forces[i, 3] += p / 2
    forces[[0, i], 1:] = forces[[i, 0], 1:]
    return forces

In [3]:
# Function to add two new elements from the selected node to the selected element
# adds two new rows to the Element matrix

def op_D0(e,n,elemat,grid):

    #create two new elements
    curr_nodes = elemat[e-1,1:3]
    new_elements = []
    #print(curr_nodes)
    i = np.shape(elemat)[0]
    for j in curr_nodes:
        if j < n:
            new_elements.append([i+1,j,n])
        else:
            new_elements.append([i+1,n,j])
        i=i+1
    #new_elements
    ele2 = np.concatenate((elemat,np.array(new_elements)))
    return ele2
        

Figure showing Operator D and T

![Operator T and D](https://raw.githubusercontent.com/syedyusuf78/Truss-Optimisation-using-RL/0ccf488e585847d543fc298b1f047e809be2b968/image.png)


In [10]:
# Function to perform operation D, add two elements to the selected nodes and an extra element to any available node.

def op_D(e,n,elemat,grid):

    ele2 = op_D0(e,n,elemat,grid)
    active_nodes,inactive_nodes = ActiveNodes(elemat,len(grid))
    curr_nodes = elemat[e-1,1:3]
    distanceArray = computeDistance(n,active_nodes,curr_nodes,grid)
    #print(distanceArray)
    #new_elements
    temp = updateLines(ele2,grid)
    el_no = np.shape(ele2)[0]
    for i in distanceArray:
        a = LineString([tuple(grid[n]),tuple(grid[i[0]])])
        intersect = filter(a.crosses,temp)
        crosses = filter(a.contains,temp)
        if len(list(intersect)+list(crosses)):
            continue
        else:
            #print("nearest node ",i[0])
            whichNodeClosest = i[0]
            if whichNodeClosest<n:
                new_element=[el_no+1,whichNodeClosest,n]
            else:
                new_element=[el_no+1,n,whichNodeClosest]
            #print("change element")
            #substitutes the selected element
            ele2 = np.r_[ele2,[new_element]]
            #print(ele2)
            break
    return sortEle(ele2)
        

In [11]:
# optimised version of Operation D
def op_D1(e,n,elemat,grid):

    ele2 = op_D0(e,n,elemat,grid)
    active_nodes,inactive_nodes = ActiveNodes(elemat,len(grid))
    curr_nodes = elemat[e-1,1:3]
    distanceArray = computeDistance(n,active_nodes,curr_nodes,grid)
    #print(distanceArray)
    #new_elements
    lines = updateLines(ele2,grid)
    el_no = np.shape(ele2)[0]
    for i in distanceArray:
        whichNodeClosest = i[0]
        if whichNodeClosest<n:
            new_element=[el_no+1,whichNodeClosest,n]
        else:
            new_element=[el_no+1,n,whichNodeClosest]
            
        a = LineString([tuple(grid[n]),tuple(grid[i[0]])])
        if not any(a.crosses(l) or a.contains(l) for l in lines):
            return sortEle(np.r_[ele2, [new_element]])
        
    return sortEle(ele2)

In [12]:
# Funtion to perform Operation T
def op_T(e,n,elemat,grid):
    
    ele2 = op_D0(e,n,elemat,grid)
    active_nodes,inactive_nodes = ActiveNodes(elemat,len(grid))
    curr_nodes = elemat[e-1,1:3]
    distanceArray = computeDistance(n,active_nodes,curr_nodes,grid)
    #print(distanceArray)
    l = LineString([tuple(grid[curr_nodes[0]]),tuple(grid[curr_nodes[1]])])
    lines = updateLines(ele2,grid)
    temp = lines
    if l in lines:
        temp.remove(l)  
    for i in distanceArray:
        a = LineString([tuple(grid[n]),tuple(grid[i[0]])])
        intersect = filter(a.crosses,temp)
        crosses = filter(a.contains,temp)
        if len(list(intersect)+list(crosses)):
            continue
        else:
            #print("nearest node ",i[0])
            whichNodeClosest = i[0]
            if whichNodeClosest<n:
                new_element=[e,whichNodeClosest,n]
            else:
                new_element=[e,n,whichNodeClosest]
            #print("change element")
            #substitutes the selected element
            ele2[e-1,:] = new_element
            break
            #print(ele2)
    lines = updateLines(ele2,grid)
    temp = lines
    el_no = np.shape(ele2)[0]
    for i in distanceArray:
        a = LineString([tuple(grid[n]),tuple(grid[i[0]])])
        intersect = filter(a.crosses,temp)
        crosses = filter(a.contains,temp)
        if len(list(intersect)+list(crosses)):
            continue
        else:
            #print("nearest node ",i[0])
            whichNodeClosest = i[0]
            if whichNodeClosest<n:
                new_element=[el_no+1,whichNodeClosest,n]
            else:
                new_element=[el_no+1,n,whichNodeClosest]
            #print("change element")
            #substitutes the selected element
            ele2 = np.r_[ele2,[new_element]]
            #print(ele2)
            break
    
    return sortEle(ele2)

In [13]:
# funtion to plot the figure
def PlotElem(elemat,grid):
    x,y = [],[]
    for e in elemat:
        for n in e[1:]:
            x.append(grid[n][0])
            y.append(grid[n][1])
        #print(x,y)
        plt.plot(x,y)
        plt.text((x[0]+x[1])/2,(y[0]+y[1])/2,e[0])
        x,y = [],[]
    plt.scatter(*zip(*grid),s=2)
    plt.show()
    #xx,yy = zip(*grid)
    #for i in range(len(grid)):
    #    plt.annotate(str(i), (xx[i] + 0.2, yy[i] + 0.2))


In [14]:
#Checking node (optimised version)
def NodeCheck1(e,n,Op,elemat,grid):
    coords_ele = [tuple(grid[elemat[e-1][1]]),tuple(grid[elemat[e-1][2]])]
    #print(coords_ele)
    coords_node = [tuple(grid[n])]
    #print(coords_node)
    lines = updateLines(elemat,grid)

    if Op == 0:
        # Check if node is within element and not crossing any other lines
        if LineString(coords_ele).contains(Point(coords_node)) and IsCrossingT(e, n, elemat, grid) == 0:
            return n, e, 0
        # Check if node is not within any other element
        elif not any(l.contains(Point(coords_node)) for l in lines) and IsCrossingT(e, n, elemat, grid) == 0:
            return n, e, 0
    elif Op == 1:
        # Check if node is not crossing any other lines
        if IsCrossingD(e, n, elemat, grid) == 0 and not any(l.contains(Point(coords_node)) for l in lines):
            return n, e, 1
            
    return tuple()

In [15]:
#Checking node 
def NodeCheck(e,n,Op,elemat,grid):
    coords_ele = [tuple(grid[elemat[e-1][1]]),tuple(grid[elemat[e-1][2]])]
    #print(coords_ele)
    coords_node = [tuple(grid[n])]
    #print(coords_node)
    lines = updateLines(elemat,grid)
    contained = filter(Point(coords_node).within, lines)
    nodeWithin = len(list(contained))

    if Point(coords_node).within(LineString(coords_ele)) and Op == 0:
        #print("node within element")
        if IsCrossingT(e,n,elemat,grid) == False :
            e1,n1,Op1 = e,n,0
            return (n1,e1,Op1)
    elif nodeWithin:
        #print("node within other elements ",nodeWithin)
        return ()

    else:
        #print("free node")
        if Op == 1:
            if IsCrossingD(e,n,elemat,grid) == 0:
                e1,n1,Op1 = e,n,Op
                return (n1,e1,Op1)
        elif Op == 0:
            if IsCrossingT(e,n,elemat,grid) == 0:
                e1,n1,Op1 = e,n,Op
                return (n1,e1,Op1)
    return tuple()

In [16]:
# funtion to check if an element is crossing any existing element
def IsCrossingD(e,n,elemat,grid):
    coords_ele = [tuple(grid[elemat[e-1][1]]),tuple(grid[elemat[e-1][2]])]
    #print(coords_ele)
    coords_node = [tuple(grid[n])]
    #print(coords_node)
    lines = updateLines(elemat,grid)
    a = LineString([tuple(grid[n]),coords_ele[0]])
    b = LineString([tuple(grid[n]),coords_ele[1]])
    contains_b = filter(b.contains, lines)
    contains_a = filter(a.contains, lines)
    cross_b = filter(b.crosses, lines)
    cross_a = filter(a.crosses, lines)
    act_nodes,inact_nodes = ActiveNodes(elemat,len(grid))
    curr_nodes = elemat[e-1,1:3]
    avail_nodes = [Point(tuple(grid[i])) for i in act_nodes if i not in curr_nodes]
    contains_nodes_a = filter(a.contains,avail_nodes)
    contains_nodes_b = filter(b.contains,avail_nodes)
    no_of_intersects = len(list(contains_a)) + len(list(contains_b)) + len(list(cross_a)) + len(list(cross_b)) + len(list(contains_nodes_b)) + len(list(contains_nodes_a))
    #print("no_of_intersects=",no_of_intersects)

    return no_of_intersects

# funtion to check if an element is crossing any existing element
def IsCrossingT(e,n,elemat,grid):

    if IsCrossingD(e,n,elemat,grid) == 0:
        active_nodes,inactive_nodes = ActiveNodes(elemat,len(grid))
        curr_nodes = elemat[e-1,1:3]
        distanceArray = computeDistance(n,active_nodes,curr_nodes,grid)
        #print(distanceArray)
        l = LineString([tuple(grid[curr_nodes[0]]),tuple(grid[curr_nodes[1]])])
        lines = updateLines(elemat,grid)
        temp = lines
        if l in temp:
            temp.remove(l)
        for i in distanceArray:
            a = LineString([tuple(grid[n]),tuple(grid[i[0]])])
            intersect = filter(a.crosses,temp)
            crosses = filter(a.contains,temp)
            if len(list(intersect)+list(crosses)):
                continue
            else:
                #print("no crossing, line can be connected to ",i[0])
                #whichNodeClosest = i[0]
                no_of_crosses = len(list(intersect)+list(crosses))
                return no_of_crosses

    return 1

In [17]:
# optimised version
def IsCrossingD1(e, n, elemat, grid):
    coords_ele = (grid[elemat[e - 1][1]], grid[elemat[e - 1][2]])
    coords_node = grid[n]
    lines = updateLines(elemat, grid)

    # Lines that contain the node
    contains_node = set(l for l in lines if coords_node in l.coords)

    # Lines that cross the line between the element nodes and the node
    line = LineString(coords_ele + (coords_node,))
    intersects = set(l for l in lines if l.intersects(line))

    # Lines that contain active nodes, excluding the nodes in the current element
    active_nodes, _ = ActiveNodes(elemat, len(grid))
    current_nodes = set(elemat[e-1,1:3])
    available_nodes = set(Point(grid[i]) for i in active_nodes if i not in current_nodes)
    contains_nodes = set(n for n in available_nodes if n.intersects(line))
    #print("hello")
    #print(len(contains_node | intersects | contains_nodes))
    return len(contains_node | intersects | contains_nodes)


In [18]:
def Avail_Nodes(elemat, grid):
    """
    Extract information about active nodes within a finite element mesh.

    Parameters:
    - elemat (list or array): A list or array containing information about elements in the mesh.
    - grid (list or array): A list or array representing the grid or nodes in the mesh.

    Returns:
    - no_nodes (int): The number of active nodes in the mesh.
    - node_map (numpy.ndarray): An array that maps original node indices to active node indices.
    """
    
    # Call the ActiveNodes function to determine active and inactive nodes
    active_nodes, inactive_nodes = ActiveNodes(elemat, len(grid))
    
    # Calculate the number of active nodes
    no_nodes = len(active_nodes)
    
    # Initialize a node map with zeros, one entry for each node in the grid
    node_map = np.zeros(len(grid), dtype=int)
    
    # Create a mapping from original node indices to active node indices
    for i in range(len(active_nodes)):
        node_map[active_nodes[i]] = i
    
    # Return the number of active nodes and the node map
    return no_nodes, node_map


In [19]:
# Finite element code

def femtruss(Ae, Le, Ee, ncon,NELEM, NNODE, F, dispID, grid, node_map,coeff,Spring):
    
    K = np.zeros((2*NNODE,2*NNODE)) # Initialize global stiffness matrix
    for ie in range(NELEM):
        eye = ncon[ie][0]
        jay = ncon[ie][1]
        # Form the transformation matrix, Lambda.
        L = Le[ie]
        A = Ae[ie]
        E = Ee[ie]
        lox,mox = (grid[jay][0]-grid[eye][0])/L,(grid[jay][1]-grid[eye][1])/L

        loy,moy = -mox,lox
        Lambda = np.array([[lox, mox, 0, 0], [0, 0, lox, mox ]])
        k = np.array([[1, -1], [-1, 1 ]]) # Local element stiffness matrix
        k = k*(A*E/L)
        #print(k)
        Lambda1 = np.transpose(Lambda)
        klocal = multi_dot([Lambda1, k, Lambda])
        #print(klocal)
        # Form ID matrix to assemble klocal into the global system
        # stiffness matrix, K.
        id1 = 2*(node_map[eye])
        id2 = id1 + 1
        id3 = 2*(node_map[jay])
        id4 = id3 + 1
        K[id1,id1] = K[id1,id1] + klocal[0,0]
        K[id1,id2] = K[id1,id2] + klocal[0,1]
        K[id2,id1] = K[id2,id1] + klocal[1,0]
        K[id2,id2] = K[id2,id2] + klocal[1,1]
        K[id1,id3] = K[id1,id3] + klocal[0,2]
        K[id1,id4] = K[id1,id4] + klocal[0,3]
        K[id2,id3] = K[id2,id3] + klocal[1,2]
        K[id2,id4] = K[id2,id4] + klocal[1,3]
        K[id3,id1] = K[id3,id1] + klocal[2,0]
        K[id3,id2] = K[id3,id2] + klocal[2,1]
        K[id4,id1] = K[id4,id1] + klocal[3,0]
        K[id4,id2] = K[id4,id2] + klocal[3,1]
        K[id3,id3] = K[id3,id3] + klocal[2,2]
        K[id3,id4] = K[id3,id4] + klocal[2,3]
        K[id4,id3] = K[id4,id3] + klocal[3,2]
        K[id4,id4] = K[id4,id4] + klocal[3,3]
    
    #Store unlaltered K as Ksing before applying the boundary conditions.
    Ksing = K
    #print(Ksing)
    for i in Spring:
        j=node_map[int(i[1])]*2 + int(i[2])
        K[j,j] += i[3]
    #print(K)
    #print(np.shape(Ksing))
    #print("global matrix",Ksing)
    #det(K)
    #detm = np.linalg.det(Ksing)
    #print("detK: ",detm)
    #if detm == 0:
        #return 0,0
    #inv(K);
    #invm = np.linalg.inv(Ksing)
    #print("invK: ,invm")
    #pause
    # Imposing displacement boundary conditions
    # ------------------------------------------
    # dispID array contains the dof which are assigned zero values.
    sn = np.shape(dispID)[0]

    Ndbc = sn
    #print(Ndbc)
    for nd in range(Ndbc):
        #print(nd)
        K = np.delete(np.delete(K,dispID[nd]-nd,0),dispID[nd]-nd,1)
        F = np.delete(F,dispID[nd]-nd,0)
    
    #print(K,F)
    
    kii = (coeff) * (np.trace(K)/np.shape(K)[0])
    #print(kii)
    np.fill_diagonal(K, np.diag(K) + kii)
    #print(K)
    
    #print(np.linalg.det(K))
    if np.linalg.det(K):
        if np.all(np.linalg.eigvals(K) > 0):
            #print("Okay till here")
            detK = np.linalg.det(K)
            #print(detK)
            # To solve for unknown displacements.
            U = np.linalg.solve(K, F)
            #SE = 0.5*U'*K*U;
            # Results
            # ---------------
            # "u" for all nodes (including those where values were specified)
            u = np.zeros((2*NNODE,1))
            for iu in range(Ndbc):
                u[dispID[iu]] = 12345.12345

            iuc = 0
            for iu in range(2*NNODE):
                if u[iu] == 12345.12345:
                    iuc = iuc+1
                else:
                    u[iu] = U[iu-iuc]

            for iu in range(Ndbc):
                u[dispID[iu]] = 0

            return u,detK
        else:
            return 0,0
        
    return 0,0

In [2]:
# finite element code

def Displacement(ele1,grid,Area,Emod,forces,dispbc,coeff,Spring):
    #PreProcessing

    #Identify the number of nodes, X and Y Coordinates of the nodes

    NNODE,node_map = Avail_Nodes(ele1,grid)
    #print(NNODE,node_map)

    NELEM = np.shape(ele1)[0]
    #print(NELEM)
    ncon = ele1[0:,1:]
    #print(ncon)
    A = Area * np.ones((NELEM,), dtype=int)
    E = Emod * np.ones((NELEM,), dtype=int)
    #print(A,E)

    #Force information
    F = np.zeros((2*NNODE,1))
    #print(F)
    Nforce = np.shape(forces)[0]
    #print(Nforce)
    for i in range(Nforce):
        dof = node_map[int(forces[i][1])]*2 + int(forces[i][2])
        F[dof] += forces[i][3]

    #print(F)

    #Displacement BCs
    Nfix = np.shape(dispbc)[0]
    #print(Nfix)
    dispID = np.array([],dtype=int)
    #print(dispID)
    for i in range(Nfix):
        dispID =  np.append(dispID, [node_map[dispbc[i][1]]*2 + dispbc[i][2]])
    dispID = np.sort(dispID)
    #print(dispID)

    #length of elements
    lines = updateLines(ele1,grid)
    L = np.array([i.length for i in lines])
    #print(L)
    #print(A,L,E,ncon,NELEM,NNODE,F,dispID,node_map)
    U,Kglobal = femtruss(A,L,E,ncon,NELEM,NNODE,F,dispID,grid,node_map,coeff,Spring)
    #print(U,Kglobal)
    #return sum(U[1::2]) if Kglobal != 0 else 10000
    u1 = U[node_map[int(forces[0][1])]*2 + int(forces[0][2])][0]
    u2 = U[node_map[int(forces[1][1])]*2 + int(forces[1][2])][0]
    return  ((u1) ** 2 + (u2) ** 2) ** 0.5 if Kglobal != 0 else 10000

In [21]:
# funtion to elvaluate if selected actions are allowable

def PossibleActions(elemat,grid):
    Actions = []
    active_nodes,inactive_nodes = ActiveNodes(elemat,len(grid))
    possible = itertools.product(inactive_nodes,range(1,np.shape(elemat)[0] + 1),[0,1])
    possible = list(possible)
    #random.shuffle(possible)
    #print(len(possible))
    for i in possible:
        n,e,Op = i
        #print(i)
        #print("e =",e, "n =",n,"Op = ",Op)
        #print("inactive_nodes=",inactive_nodes)
        #print("active_nodes=",active_nodes)
        if (n,e,Op) == NodeCheck(e,n,Op,elemat,grid):
            Actions.append([n,e,Op])
        #else:
        #print(len(possible))
    #print(Actions)
    #print("No of possible actions: ",len(Actions))
    return Actions

In [22]:
# funtion to sort the element matrix
def sortEle(a):
    A = a[np.lexsort((a[:,2], a[:, 1])), :]
    C = [i for i in range(1,np.shape(A)[0]+1)]
    A[:,0] = C
    #print(A)
    return A

## END